In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from mlagents_envs.environment import UnityEnvironment

In [2]:
if(torch.cuda.is_available()):
    device = torch.device("cuda")
    print(device, torch.cuda.get_device_name(0))
else:
    device= torch.device("cpu")
    print(device)

cpu


C:\Users\admin\Anaconda3\envs\PyTorch-MLAgent\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
N_STATES  = 8
N_ACTIONS =2

In [4]:
class Net(nn.Module):
    def __init__(self, ):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(N_STATES, 128)
        self.layer1.weight.data.normal_(0, 0.2)   # initialization
        self.layer2 = nn.Linear(128, 128)
        self.layer2.weight.data.normal_(0, 0.2)   # initialization
        self.out = nn.Linear(128, N_ACTIONS)
        self.out.weight.data.normal_(0, 0.2)   # initialization

    def forward(self, x):
        x = self.layer1(x)
        x = F.relu(x)
        x = self.layer2(x)
        x = F.relu(x)
        actions_value = self.out(x)
        return actions_value

In [5]:
net = Net().to(device)

### Connect to Unity

In [6]:
env = UnityEnvironment(file_name= None, base_port=5004)

#Go to Unity and press Play to start

This means that some features may not work unless you upgrade the package with the lower version.Please find the versions that work best together from our release page.
https://github.com/Unity-Technologies/ml-agents/releases


In [7]:
env.reset()

In [8]:
behaviorNames = list(env.behavior_specs.keys())
behaviorName = behaviorNames[0]
behavior_spec = env.behavior_specs[behaviorName]
print(behaviorNames)
print(behavior_spec)

['3DBall?team=0']
BehaviorSpec(observation_shapes=[(8,)], action_spec=ActionSpec(continuous_size=2, discrete_branches=()))


In [9]:
DecisionSteps, TerminalSteps = env.get_steps(behaviorName)
print(DecisionSteps.agent_id)
print(TerminalSteps.agent_id)

[0 1 2]
[]


In [10]:
DecisionAgentIDs = list(DecisionSteps.agent_id)
TerminalAgentIDs = list(TerminalSteps.agent_id)
print(DecisionAgentIDs, TerminalAgentIDs)

[0, 1, 2] []


### Send decision steps to NN to get action

In [11]:
DecisionSteps.obs

[array([[-0.01467304, -0.01468306, -0.5208206 ,  4.        , -0.79952097,
          0.        ,  0.        ,  0.        ],
        [-0.02614026,  0.03401016, -0.45768166,  4.        , -0.0055027 ,
          0.        ,  0.        ,  0.        ],
        [ 0.06363224,  0.03799658, -1.1360741 ,  4.        , -0.4150591 ,
          0.        ,  0.        ,  0.        ]], dtype=float32)]

In [12]:
states = DecisionSteps.obs[0]
print(states)

[[-0.01467304 -0.01468306 -0.5208206   4.         -0.79952097  0.
   0.          0.        ]
 [-0.02614026  0.03401016 -0.45768166  4.         -0.0055027   0.
   0.          0.        ]
 [ 0.06363224  0.03799658 -1.1360741   4.         -0.4150591   0.
   0.          0.        ]]


In [13]:
states = torch.FloatTensor(states).to(device)

In [14]:
actions = net(states)
print(actions)

tensor([[-1.1012, -0.6932],
        [-0.7616, -0.6634],
        [-1.0846, -0.8035]], grad_fn=<AddmmBackward>)


In [15]:
actions = actions.cpu().detach().numpy()
print(actions)

[[-1.1011918  -0.69320214]
 [-0.7616119  -0.66337574]
 [-1.084576   -0.8035136 ]]


In [16]:
env.set_actions(behaviorName, actions)

In [17]:
env.step()

In [18]:
env.close()

# Play for N steps

In [19]:
env = UnityEnvironment(file_name= None, base_port=5004)

This means that some features may not work unless you upgrade the package with the lower version.Please find the versions that work best together from our release page.
https://github.com/Unity-Technologies/ml-agents/releases


In [20]:
env.reset()
behaviorNames = list(env.behavior_specs.keys())
behaviorName = behaviorNames[0]

In [21]:
for frame in range(100):
    DecisionSteps, TerminalSteps = env.get_steps(behaviorName)
    for AgentID in TerminalSteps.agent_id:
        print("step", frame, "agent ", AgentID, "has terminal step")
    
    state = DecisionSteps.obs[0]
    state = torch.FloatTensor(state).to(device)
    action = net(state)
    action = action.cpu().detach().numpy()
    env.set_actions(behaviorName, action)   
    env.step()

step 22 agent  2 has terminal step
step 38 agent  1 has terminal step
step 49 agent  0 has terminal step
step 61 agent  2 has terminal step
step 70 agent  1 has terminal step
step 83 agent  0 has terminal step
step 92 agent  2 has terminal step
step 97 agent  1 has terminal step


In [22]:
env.close()